### 1. 데이터 로드,  초기 경로 설정

In [5]:
import os
import warnings
import tqdm
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
from sklearn.preprocessing import scale

In [6]:
%load_ext autoreload
%autoreload 2

import sys

# 현재 폴더의 상위 상위 폴더 경로를 가져옵니다.
parent_dir = os.path.dirname(os.getcwd())
grandparents_dir = os.path.dirname(parent_dir)

# 상위 상위 폴더 경로를 sys.path에 추가합니다.
sys.path.insert(0, parent_dir)
sys.path.insert(0, grandparents_dir)

# 패키지 경로를 직접 import합니다.
from socceraction.data.statsbomb import StatsBombLoader
import socceraction.spadl as spadl
import socceraction.vaep.formula as vaepformula

In [7]:
import csv
import tensorflow as tf
import os
import scipy.io as sio
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from nn.td_two_tower_lstm import TD_Prediction_TT_Embed
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from utils import handle_trace_length, get_together_training_batch, compromise_state_trace_length
from configuration import MODEL_TYPE, MAX_TRACE_LENGTH, FEATURE_NUMBER, BATCH_SIZE, GAMMA, H_SIZE, \
    model_train_continue, FEATURE_TYPE, ITERATE_NUM, learning_rate, SPORT, save_mother_dir, hidden_dim, lr, gamma, batch_size, memory_size, max_trace_length, output_dim, num_layers

2024-08-21 15:29:40.203695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 15:29:40.224701: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 15:29:40.232936: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-21 15:29:40.256367: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-21 15:29:41.563854: W tensorflow/compiler/tf2

In [8]:
gim_folder = os.path.dirname(os.getcwd())

LOG_DIR = str(gim_folder) + save_mother_dir + "/models/hybrid_sl_log_NN/Scale-three-cut_together_log_train_feature" + str(
    FEATURE_TYPE) + "_batch" + str(
    BATCH_SIZE) + "_iterate" + str(
    ITERATE_NUM) + "_lr" + str(
    learning_rate) + "_" + str(MODEL_TYPE) + "_MaxTL" + str(MAX_TRACE_LENGTH)
SAVED_NETWORK = str(gim_folder) + save_mother_dir + "/models/hybrid_sl_saved_NN/Scale-three-cut_together_saved_networks_feature" + str(
    FEATURE_TYPE) + "_batch" + str(
    BATCH_SIZE) + "_iterate" + str(
    ITERATE_NUM) + "_lr" + str(
    learning_rate) + "_" + str(MODEL_TYPE) + "_MaxTL" + str(MAX_TRACE_LENGTH)

In [9]:
DATA_STORE = "../datastore"
DIR_GAMES_ALL = os.listdir(DATA_STORE)
number_of_total_game = len(DIR_GAMES_ALL)

display(f"총 게임 수 : {number_of_total_game}")

'총 게임 수 : 64'

### 2. Train_network와 Cost/Plot 함수 저장

In [10]:
# Cost 저장
def write_game_average_csv(data_record):
    """
    write the cost of training
    :param data_record: the recorded cost dict
    """
    try:
        if os.path.exists(LOG_DIR + '/avg_cost_record.csv'):
            with open(LOG_DIR + '/avg_cost_record.csv', 'a') as csvfile:
                fieldnames = (data_record[0]).keys()
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                for record in data_record:
                    writer.writerow(record)
        else:
            with open(LOG_DIR + '/avg_cost_record.csv', 'w') as csvfile:
                fieldnames = (data_record[0]).keys()
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                for record in data_record:
                    writer.writerow(record)
    except:
        if os.path.exists(LOG_DIR + '/avg_cost_record2.csv'):
            with open(LOG_DIR + '/avg_cost_record.csv', 'a') as csvfile:
                fieldnames = (data_record[0]).keys()
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                for record in data_record:
                    writer.writerow(record)
        else:
            with open(LOG_DIR + '/avg_cost_record2.csv', 'w') as csvfile:
                fieldnames = (data_record[0]).keys()
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                for record in data_record:
                    writer.writerow(record)

In [11]:
def plot_qvalue_goal(GTR, GD, SHOT_TRY, state, y, game_id, ITER):
    print(f"len gtr : {len(GTR)}")
    print(f"len GD : {len(GD)}")
    print(f"len y : {len(y)}")
    
    fig, ax1 = plt.subplots(figsize=(15, 10))
    ax1.set_xlabel('GTR')
    ax1.set_ylabel('Prob')
    line1 = ax1.plot(GTR, y[:,0], 'red', label='Home', linestyle="--")
    line2 = ax1.plot(GTR, y[:,1], 'blue', label='Away', linestyle="--")
    line3 = ax1.plot(GTR, y[:,2], 'yellow', label='Neither', linestyle="--")

    ax2 = ax1.twinx()
    ax2.set_ylabel('GD')
    line4 = ax2.plot(GTR, GD, 'deeppink', label='Goal Difference')

    lines = line1 + line2 + line3 + line4
    labels = [l.get_label() for l in lines]
    ax1.legend(lines, labels, loc='upper right')

    plt.gca().invert_xaxis()  # x축 반전

    # 샷이 일어난 순간을 수직선으로 표시
    for shot in SHOT_TRY:
        color = 'red' if shot[1] == 1 else 'blue'
        shot_team = 'H' if shot[1] == 1 else 'A'
        plt.vlines(x=shot[0], ymin=-0.001, ymax=0.003, color=color, linewidth=2, alpha=0.7)
        plt.text(shot[0], 0.0035, shot_team, color=color, fontsize=12, ha='center')

    plt.title(f'{game_id} Goal Probability')
    plt.savefig(f'./plot/goal_prob/{game_id}_{ITER}.jpg')



def game_plot(FEATURE_NUMBER, hidden_dim, MAX_TRACE_LENGTH, learning_rate, SAVED_NETWORK, SPORT, MODEL_VERSION, GAME_ID):
    # loading network
    # 모델 불러오기
    model = TD_Prediction_TT_Embed(FEATURE_NUMBER, hidden_dim, MAX_TRACE_LENGTH, learning_rate)
    check_path = os.path.join(SAVED_NETWORK, f"{SPORT}-game-{MODEL_VERSION}.pt")
    checkpoint = torch.load(check_path)  # Load checkpoint
    model.load_state_dict(checkpoint['model_state_dict'])
    model.optimizer_home.load_state_dict(checkpoint['home_optimizer_state_dict'])
    model.optimizer_away.load_state_dict(checkpoint['away_optimizer_state_dict'])
    check_point_game_number = checkpoint['check_point_game_number']

    # 데이터 로드
    game_id = GAME_ID

    DATA_STORE = "../datastore"
    game_list = []
    trace_length_list = []
    reward_list = []

    # load data
    dir_game = f'{game_id}'
    state = sio.loadmat(DATA_STORE + "/" + dir_game + "/" + 'rnn_input')
    reward = sio.loadmat(DATA_STORE + "/" + dir_game + "/" + 'rnn_reward')
    trace_length = sio.loadmat(DATA_STORE + "/" + dir_game + "/" + 'trace')
    game_list += (state['data'])[0].tolist()
    reward_list += (reward['data'])[0].tolist()
    trace_length_list += (trace_length['data'])[0].tolist()
    
    
    GTR = []
    GD = []
    SHOT_TRY = []
    for state in game_list:
        GTR.append(state[-1][0])
        GD.append(state[-1][4])
        if state[-1][22] == 1 or state[-1][29] == 1 or state[-1][32] == 1:
            SHOT_TRY.append([state[-1][0], round(state[-1][10])])
        if state[-1][31] == 1 :
            SHOT_TRY.append([state[-1][0], -round(state[-1][10])])        


    home_away_indicator = [1 if state[-1][10] > 1 else 0 for state in game_list]
    # Prediction
    state_trace_length, state_input, reward = compromise_state_trace_length(trace_length_list, game_list, reward_list, MAX_TRACE_LENGTH)


    # get the batch variables
    y_batch = []

    # Target 값 계산
    trace_batch_tensor = torch.tensor(state_trace_length, dtype=torch.int32)
    s_batch_tensor = torch.tensor(state_input, dtype=torch.float32)
    home_away_indicator_tensor = torch.tensor(home_away_indicator, dtype=torch.bool)
    # 모델을 평가 모드로 전환
    model.eval()
    # forward를 통해 출력 계산
    with torch.no_grad():
        outputs_t0 = model.forward(s_batch_tensor, trace_batch_tensor, home_away_indicator_tensor)
    # 필요 시 numpy 배열로 변환 (TensorFlow의 sess.run()과 동일한 역할)
    readout_t1_batch = outputs_t0.numpy()
    plot_qvalue_goal(GTR, GD, SHOT_TRY, np.array(game_list), readout_t1_batch, game_id, check_point_game_number)

In [12]:
def train_network(model):
    """
    training thr neural network game by game
    :param sess: session of tf
    :param model: nn model
    :return:
    """
    game_number = 0
    global_counter = 0
    converge_flag = False

    # Check if CUDA is available and use GPU if possible
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # model.to(device)

    # instance of Pytorch Model
    writer = SummaryWriter(LOG_DIR)

    # loading network
    if model_train_continue:
        check_path = os.path.join(SAVED_NETWORK, f"{SPORT}-game-{1900}.pt")
        checkpoint = torch.load(check_path)  # Load checkpoint
        model.load_state_dict(checkpoint['model_state_dict'])
        model.optimizer_home.load_state_dict(checkpoint['home_optimizer_state_dict'])
        model.optimizer_away.load_state_dict(checkpoint['away_optimizer_state_dict'])
        check_point_game_number = checkpoint['check_point_game_number']
        epoch = checkpoint['epoch']
        game_number_checkpoint = check_point_game_number % number_of_total_game
        game_number = check_point_game_number
        game_starting_point = 0
        print(f"Interation Now : {epoch}")
        print("Successfully loaded:", SAVED_NETWORK)
    else:
        print("Could not find old network weights")

    game_diff_record_all = []

    # 모든 게임 - 30 iteration
    while True:
        game_diff_record_dict = {}
        iteration_now = game_number / number_of_total_game + 1
        game_diff_record_dict.update({"Iteration": iteration_now})
        if converge_flag:
            break
        elif game_number >= number_of_total_game * ITERATE_NUM:
            break
        else:
            converge_flag = True
        # 게임 불러오기
        for dir_game in DIR_GAMES_ALL:

            if model_train_continue:
                if checkpoint and 135:  # go the check point data
                    game_starting_point += 1
                    if game_number_checkpoint + 1 > game_starting_point:
                        continue

            v_diff_record = []
            game_number += 1
            game_cost_record = []
            game_files = os.listdir(DATA_STORE + "/" + dir_game)
            # 게임 안의 episode 별로 reward, input, trace가 저장되어 있다.
            for filename in game_files:
                if "rnn_reward" in filename:
                    reward_name = filename
                elif "rnn_input" in filename:
                    state_input_name = filename
                elif "trace" in filename:
                    state_trace_length_name = filename

            reward = sio.loadmat(DATA_STORE + "/" + dir_game + "/" + reward_name)
            reward = (reward['data'])[0]

            try:
                print(f"{dir_game} episode length : {len(reward)}")
            except:
                print("\n" + dir_game)
                raise ValueError("reward wrong")
            
            # state_input 로드
            state_input = sio.loadmat(DATA_STORE + "/" + dir_game + "/" + state_input_name)
            state_input = (state_input['data'])[0]
            state_trace_length = sio.loadmat(DATA_STORE + "/" + dir_game + "/" + state_trace_length_name)
            state_trace_length = (state_trace_length['data'])[0]
            state_trace_length, state_input, reward = compromise_state_trace_length(state_trace_length, state_input,
                                                                                    reward, MAX_TRACE_LENGTH)

            reward_count = len(reward)
            print("reward number" + str(reward_count))
            print("=> load file" + str(dir_game) + " success")
            if len(state_input) != len(reward) or len(state_trace_length) != len(reward):
                raise Exception('state length does not equal to reward length')

            train_len = len(state_input)
            train_number = 0
            s_t0 = state_input[train_number]
            train_number += 1

            while True:
                # try:
                batch_return, train_number, s_tl, home_away_indicator = get_together_training_batch(s_t0,
                                                                                    state_input,
                                                                                    reward,
                                                                                    train_number,
                                                                                    train_len,
                                                                                    state_trace_length,
                                                                                    BATCH_SIZE)

                # Convert lists to numpy arrays first
                s_t0_batch = np.array([d[0] for d in batch_return], dtype=np.float32)
                s_t1_batch = np.array([d[1] for d in batch_return], dtype=np.float32)
                r_t_batch = np.array([d[2] for d in batch_return], dtype=np.float32)
                trace_t0_batch = np.array([d[3] for d in batch_return], dtype=np.int32)
                trace_t1_batch = np.array([d[4] for d in batch_return], dtype=np.int32)
                y_batch = []

                # Target 값 계산
                trace_t1_batch_tensor = torch.tensor(trace_t1_batch, dtype=torch.int32)
                s_t1_batch_tensor = torch.tensor(s_t1_batch, dtype=torch.float32)
                home_away_indicator_tensor = torch.tensor(home_away_indicator, dtype=torch.bool)
                # forward pass를 통해 출력 계산
                with torch.no_grad():
                    outputs_t1 = model.forward(s_t1_batch_tensor, trace_t1_batch_tensor, home_away_indicator_tensor[:, 1])
                # 홈/어웨이 출력 선택에 따라 readout_t1_batch 결정
                # readout_t1_batch = torch.where(home_away_indicator_tensor[:, 1], outputs_t1[:, 0], outputs_t1[:, 1])

                # 필요 시 numpy 배열로 변환 (TensorFlow의 sess.run()과 동일한 역할)
                outputs_t1 = outputs_t1.detach()
                readout_t1_batch = outputs_t1.numpy()

                # print(f"readout_t1_batch : {readout_t1_batch}")

                for i in range(0, len(batch_return)):
                    terminal = batch_return[i][5]
                    cut = batch_return[i][6]
                    # if terminal, only equals reward
                    if terminal or cut:
                        y_home = float((r_t_batch[i])[0])
                        y_away = float((r_t_batch[i])[1])
                        y_end = float((r_t_batch[i])[2])
                        y_batch.append([y_home, y_away, y_end])
                        break
                    else:
                        y_home = float((r_t_batch[i])[0]) + GAMMA * (readout_t1_batch[i]).tolist()[0]
                        y_away = float((r_t_batch[i])[1]) + GAMMA * (readout_t1_batch[i]).tolist()[1]
                        y_end = float((r_t_batch[i])[2]) + GAMMA * (readout_t1_batch[i]).tolist()[2]

                        wandb.log({"Home_prob": (readout_t1_batch[i]).tolist()[0]})
                        wandb.log({"Away_prob": (readout_t1_batch[i]).tolist()[1]})
                        wandb.log({"End_prob": (readout_t1_batch[i]).tolist()[2]})
                        # print(f"no terminal or cut : {[y_home, y_away, y_end]}")
                        y_batch.append([y_home, y_away, y_end])

                # y_batch를 텐서로 변환
                y_batch_tensor = torch.tensor(y_batch, dtype=torch.float32)
                # trace_t0_batch와 s_t0_batch를 텐서로 변환
                trace_t0_batch_tensor = torch.tensor(trace_t0_batch, dtype=torch.int32)
                s_t0_batch_tensor = torch.tensor(s_t0_batch, dtype=torch.float32)

                # s_t0 - forward | 손실 계산
                home_loss, away_loss, read_out = model.train_step(s_t0_batch_tensor, trace_t0_batch_tensor, home_away_indicator_tensor[:, 0], y_batch_tensor)
                
                # 출력 및 디버깅 정보
                diff = torch.mean(torch.abs(y_batch_tensor - read_out)).item()
                cost_out = torch.mean(torch.square(y_batch_tensor - read_out)).item()
                wandb.log({"td_diff": diff})
                wandb.log({"td_cost": cost_out})

                v_diff_record.append(diff)

                if cost_out > 0.0001:
                    converge_flag = False
                global_counter += 1
                game_cost_record.append(cost_out)
                # global_step 및 summary_train에 해당하는 값을 TensorBoard에 기록
                writer.add_scalar('Loss/train', cost_out, global_step=global_counter)
                writer.add_scalar('Diff/train', diff, global_step=global_counter)
                s_t0 = s_tl

                # print info
                if terminal or ((train_number - 1) / BATCH_SIZE) % 5 == 1:
                    print("TIMESTEP:", train_number, "Game:", game_number)
                    home_avg = sum(read_out[:, 0]) / len(read_out[:, 0])
                    away_avg = sum(read_out[:, 1]) / len(read_out[:, 1])
                    end_avg = sum(read_out[:, 2]) / len(read_out[:, 2])
                    print("home average:{0}, away average:{1}, end average:{2}".format(str(home_avg), str(away_avg),
                                                                                       str(end_avg)))
                    print("cost of the network is" + str(cost_out))

                if terminal:
                    v_diff_record_average = sum(v_diff_record) / len(v_diff_record)
                    game_diff_record_dict.update({dir_game: v_diff_record_average})


                    if game_number % 100 == 0:
                        # save progress after a game
                        save_path = os.path.join(SAVED_NETWORK, f"{SPORT}-game-{game_number}.pt")
                        # 모델의 상태 딕셔너리(가중치 등)를 저장
                        torch.save({
                            'model_state_dict': model.state_dict(),
                            'home_optimizer_state_dict' : model.optimizer_home.state_dict(),
                            'away_optimizer_state_dict' : model.optimizer_away.state_dict(),
                            'check_point_game_number': game_number,
                            'global_step': global_counter,
                            'epoch' : iteration_now
                        }, save_path)
                        # plot 그리기
                        for game_id in [7537, 7550, 8649]:
                            game_plot(FEATURE_NUMBER, hidden_dim, MAX_TRACE_LENGTH, learning_rate, SAVED_NETWORK, SPORT, game_number, game_id)
                    break

            cost_per_game_average = sum(game_cost_record) / len(game_cost_record)
            write_game_average_csv([{"iteration": str(game_number / number_of_total_game + 1), "game": game_number,
                                     "cost_per_game_average": cost_per_game_average}])

        game_diff_record_all.append(game_diff_record_dict)

    # 마지막 모델
    # save progress after a game
    save_path = os.path.join(SAVED_NETWORK, f"{SPORT}-game-{game_number}.pt")
    # 모델의 상태 딕셔너리(가중치 등)를 저장
    torch.save({
        'model_state_dict': model.state_dict(),
        'home_optimizer_state_dict' : model.optimizer_home.state_dict(),
        'away_optimizer_state_dict' : model.optimizer_away.state_dict(),
        'check_point_game_number': game_number,
        'global_step': global_counter,
        'epoch' : iteration_now
    }, save_path)
    # plot 그리기
    for game_id in [7537, 7550, 8649]:
        game_plot(FEATURE_NUMBER, hidden_dim, MAX_TRACE_LENGTH, learning_rate, SAVED_NETWORK, SPORT, game_number, game_id)


In [13]:
def train_start():

    if not os.path.exists(LOG_DIR):
        os.makedirs(LOG_DIR)
    if not os.path.exists(SAVED_NETWORK):
        os.makedirs(SAVED_NETWORK)


    if MODEL_TYPE == "two_tower":
        model = TD_Prediction_TT_Embed(FEATURE_NUMBER, hidden_dim, MAX_TRACE_LENGTH, learning_rate)
    else:
        raise ValueError("MODEL_TYPE error")
    train_network(model)

### 3. 학습 시작

In [ ]:
if __name__ == '__main__':
    train_start()